# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [78]:
import time

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [4]:
driver = webdriver.Chrome()
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [14]:
search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", search_box)
search_box.click()

In [16]:
text_input = driver.find_element_by_name('MineId')
text_input.send_keys('3503598')

In [17]:
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [20]:
driver.find_element_by_name('BDate').send_keys('1/1/1995')

In [23]:
violations_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
driver.execute_script("arguments[0].scrollIntoView(true)", violations_button)
violations_button.click()

In [24]:
submit_button = driver.find_element_by_name('submit')
driver.execute_script("arguments[0].scrollIntoView(true)", submit_button)
submit_button.click()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [64]:
results = driver.find_elements_by_xpath('//*[@id="content"]/table[5]/tbody/tr')

violations = []

for result in results[1:]: 
    
    data = {}
    if len(result.find_elements_by_class_name('drsviols')) != 0:
        print(result.find_elements_by_class_name('drsviols')[0].text)
        data['violator'] = result.find_elements_by_class_name('drsviols')[0].text
        print(result.find_elements_by_class_name('drsviols')[1].text)
        data['contract_id'] = result.find_elements_by_class_name('drsviols')[1].text
        print(result.find_elements_by_class_name('drsviols')[2].text)
        data['citation_no'] = result.find_elements_by_class_name('drsviols')[2].text
        print(result.find_elements_by_class_name('drsviols')[3].text)
        data['case_no'] = result.find_elements_by_class_name('drsviols')[3].text
        print(result.find_elements_by_class_name('drsviols')[4].text)
        data['date_issues'] = result.find_elements_by_class_name('drsviols')[4].text
        print(result.find_elements_by_class_name('drsviols')[5].text)
        data['final_order_date'] = result.find_elements_by_class_name('drsviols')[5].text
        print(result.find_elements_by_class_name('drsviols')[6].text)
        data['section_of_act'] = result.find_elements_by_class_name('drsviols')[6].text
        print(result.find_elements_by_class_name('drsviols')[7].text)
        data['date_terminated'] = result.find_elements_by_class_name('drsviols')[7].text
        print(result.find_elements_by_class_name('drsviols')[8].text)
        data['citation'] = result.find_elements_by_class_name('drsviols')[8].text
        print(result.find_elements_by_class_name('drsviols')[9].text)
        data['s_and_s'] = result.find_elements_by_class_name('drsviols')[9].text
        print(result.find_elements_by_class_name('drsviols')[10].text)
        data['standard'] = result.find_elements_by_class_name('drsviols')[10].text
        print(result.find_elements_by_class_name('drsviols')[10].find_element_by_tag_name('a').get_attribute('href'))
        data['standard_url'] = result.find_elements_by_class_name('drsviols')[10].find_element_by_tag_name('a').get_attribute('href')
        data['proposed_penalty'] = result.find_elements_by_class_name('drsviols')[11].text
        data['citation_status'] = result.find_elements_by_class_name('drsviols')[12].text
        data['current_penalty'] = result.find_elements_by_class_name('drsviols')[13].text
        data['amount_paid'] = result.find_elements_by_class_name('drsviols')[14].text
        
        violations.append(data)
        

Newberg Rock & Dirt
  
8992368  
000440809
5/2/2017
7/23/2017 
104(a)
5/2/2017
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
8992214  
000435717
2/7/2017
5/21/2017 
104(a)
2/8/2017
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
8872461  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.12004
http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-vol1/pdf/CFR-2015-title30-vol1-sec56-12004.pdf
Newberg Rock & Dirt
  
8872462  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-vol1/pdf/CFR-2015-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
8790965  
000383840
4/15/2015
7/17/2015 
104(a)
4/15/2015
C
N
56.13015
http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-vol1/pdf/CFR-2015-title30-vol1-sec56-13015.pdf
Newberg Rock & Dirt
  
8790964  
000383840
4/13/2015
7/1

Newberg Rock & Dirt
  
6342895  
000121048
5/23/2007
8/17/2007 
104(a)
5/24/2007
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2007-title30-vol1/pdf/CFR-2007-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
6383438  
000086543
3/1/2006
6/16/2006 
104(a)
3/2/2006
C
N
56.11002
http://www.gpo.gov/fdsys/pkg/CFR-2006-title30-vol1/pdf/CFR-2006-title30-vol1-sec56-11002.pdf
Newberg Rock & Dirt
  
6383437  
000086543
3/1/2006
6/16/2006 
104(a)
3/2/2006
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2006-title30-vol1/pdf/CFR-2006-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
6368407  
000051247
10/27/2004
7/23/2005 
104(a)
1/21/2005
C
N
62.120
http://www.gpo.gov/fdsys/pkg/CFR-2004-title30-vol1/pdf/CFR-2004-title30-vol1-sec62-120.pdf
Newberg Rock & Dirt
  
6368406  
000044260
10/25/2004
1/12/2005 
104(a)
10/27/2004
C
Y
56.14131(a)
http://www.gpo.gov/fdsys/pkg/CFR-2004-title30-vol1/pdf/CFR-2004-title30-vol1-sec56-14131.pdf


### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [65]:
import pandas as pd

In [69]:
df = pd.DataFrame(violations)
df

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.00,000440809,C,8992368,Closed,,116.00,5/2/2017,5/2/2017,7/23/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
1,116.00,000435717,C,8992214,Closed,,116.00,2/7/2017,2/8/2017,5/21/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
2,100.00,000398910,C,8872461,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
3,100.00,000398910,C,8872462,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
4,100.00,000383840,C,8790965,Closed,,100.00,4/15/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
5,100.00,000383840,C,8790964,Closed,,100.00,4/13/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.4101,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
6,100.00,000383840,C,8790962,Closed,,100.00,4/10/2015,4/10/2015,7/17/2015,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
7,873.00,000383840,C,8790963,Closed,,873.00,4/10/2015,4/13/2015,7/17/2015,873.00,Y,104(a),56.15020,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
8,100.00,000345066,C,8694457,Closed,,100.00,1/29/2014,1/29/2014,4/12/2014,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Newberg Rock & Dirt
9,100.00,000345066,C,8694458,Closed,,100.00,1/29/2014,1/29/2014,4/12/2014,100.00,N,104(a),56.12018,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Newberg Rock & Dirt


In [70]:
df.to_csv("3503598-violations.csv", index=False)

## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

In [71]:
# accessing the website 
driver = webdriver.Chrome()
driver.get("https://arlweb.msha.gov/drs/drshome.htm")


# searching for a mine
search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", search_box)
search_box.click()

text_input = driver.find_element_by_name('MineId')
text_input.send_keys('3503598')

search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()


time.sleep(1)


# finding reports 

driver.find_element_by_name('BDate').send_keys('1/1/1995')

violations_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
driver.execute_script("arguments[0].scrollIntoView(true)", violations_button)
violations_button.click()

submit_button = driver.find_element_by_name('submit')
driver.execute_script("arguments[0].scrollIntoView(true)", submit_button)
submit_button.click()


time.sleep(1)


# saving reports 

results = driver.find_elements_by_xpath('//*[@id="content"]/table[5]/tbody/tr')

violations = []

for result in results[1:]: 
    
    data = {}
    if len(result.find_elements_by_class_name('drsviols')) != 0:
        print(result.find_elements_by_class_name('drsviols')[0].text)
        data['violator'] = result.find_elements_by_class_name('drsviols')[0].text
        print(result.find_elements_by_class_name('drsviols')[1].text)
        data['contract_id'] = result.find_elements_by_class_name('drsviols')[1].text
        print(result.find_elements_by_class_name('drsviols')[2].text)
        data['citation_no'] = result.find_elements_by_class_name('drsviols')[2].text
        print(result.find_elements_by_class_name('drsviols')[3].text)
        data['case_no'] = result.find_elements_by_class_name('drsviols')[3].text
        print(result.find_elements_by_class_name('drsviols')[4].text)
        data['date_issues'] = result.find_elements_by_class_name('drsviols')[4].text
        print(result.find_elements_by_class_name('drsviols')[5].text)
        data['final_order_date'] = result.find_elements_by_class_name('drsviols')[5].text
        print(result.find_elements_by_class_name('drsviols')[6].text)
        data['section_of_act'] = result.find_elements_by_class_name('drsviols')[6].text
        print(result.find_elements_by_class_name('drsviols')[7].text)
        data['date_terminated'] = result.find_elements_by_class_name('drsviols')[7].text
        print(result.find_elements_by_class_name('drsviols')[8].text)
        data['citation'] = result.find_elements_by_class_name('drsviols')[8].text
        print(result.find_elements_by_class_name('drsviols')[9].text)
        data['s_and_s'] = result.find_elements_by_class_name('drsviols')[9].text
        print(result.find_elements_by_class_name('drsviols')[10].text)
        data['standard'] = result.find_elements_by_class_name('drsviols')[10].text
        print(result.find_elements_by_class_name('drsviols')[10].find_element_by_tag_name('a').get_attribute('href'))
        data['standard_url'] = result.find_elements_by_class_name('drsviols')[10].find_element_by_tag_name('a').get_attribute('href')
        data['proposed_penalty'] = result.find_elements_by_class_name('drsviols')[11].text
        data['citation_status'] = result.find_elements_by_class_name('drsviols')[12].text
        data['current_penalty'] = result.find_elements_by_class_name('drsviols')[13].text
        data['amount_paid'] = result.find_elements_by_class_name('drsviols')[14].text
        
        violations.append(data)
        

# saving the data

df = pd.DataFrame(violations)
df.to_csv("3503598-violations.csv", index=False)

Newberg Rock & Dirt
  
8992368  
000440809
5/2/2017
7/23/2017 
104(a)
5/2/2017
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
8992214  
000435717
2/7/2017
5/21/2017 
104(a)
2/8/2017
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
8872462  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-vol1/pdf/CFR-2015-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
8872461  
000398910
10/21/2015
1/13/2016 
104(a)
10/27/2015
C
N
56.12004
http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-vol1/pdf/CFR-2015-title30-vol1-sec56-12004.pdf
Newberg Rock & Dirt
  
8790965  
000383840
4/15/2015
7/17/2015 
104(a)
4/15/2015
C
N
56.13015
http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-vol1/pdf/CFR-2015-title30-vol1-sec56-13015.pdf
Newberg Rock & Dirt
  
8790964  
000383840
4/13/2015
7/1

Newberg Rock & Dirt
  
6342894  
000121048
5/23/2007
8/17/2007 
104(a)
5/24/2007
C
N
56.14132(a)
http://www.gpo.gov/fdsys/pkg/CFR-2007-title30-vol1/pdf/CFR-2007-title30-vol1-sec56-14132.pdf
Newberg Rock & Dirt
  
6383438  
000086543
3/1/2006
6/16/2006 
104(a)
3/2/2006
C
N
56.11002
http://www.gpo.gov/fdsys/pkg/CFR-2006-title30-vol1/pdf/CFR-2006-title30-vol1-sec56-11002.pdf
Newberg Rock & Dirt
  
6383437  
000086543
3/1/2006
6/16/2006 
104(a)
3/2/2006
C
N
56.14107(a)
http://www.gpo.gov/fdsys/pkg/CFR-2006-title30-vol1/pdf/CFR-2006-title30-vol1-sec56-14107.pdf
Newberg Rock & Dirt
  
6368407  
000051247
10/27/2004
7/23/2005 
104(a)
1/21/2005
C
N
62.120
http://www.gpo.gov/fdsys/pkg/CFR-2004-title30-vol1/pdf/CFR-2004-title30-vol1-sec62-120.pdf
Newberg Rock & Dirt
  
6368406  
000044260
10/25/2004
1/12/2005 
104(a)
10/27/2004
C
Y
56.14131(a)
http://www.gpo.gov/fdsys/pkg/CFR-2004-title30-vol1/pdf/CFR-2004-title30-vol1-sec56-14131.pdf


# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [74]:
df_mines = pd.read_csv("mines-subset.csv")
df_mines

,id
0,4104757
1,801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

In [75]:
df_mines = pd.read_csv("mines-subset.csv", dtype='str')
df_mines

,id
0,4104757
1,0801306
2,3609931


### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

In [83]:
def get_mines_violations(row): 
    
    # accessing the website 
    driver = webdriver.Chrome()
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")


    # searching for a mine
    search_box = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[1]')
    driver.execute_script("arguments[0].scrollIntoView(true)", search_box)
    search_box.click()

    text_input = driver.find_element_by_name('MineId')
    text_input.send_keys(row['id'])
    print(row['id'])

    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()

    time.sleep(1)

    # finding reports 

    driver.find_element_by_name('BDate').send_keys('1/1/1995')

    violations_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
    driver.execute_script("arguments[0].scrollIntoView(true)", violations_button)
    violations_button.click()

    submit_button = driver.find_element_by_name('submit')
    driver.execute_script("arguments[0].scrollIntoView(true)", submit_button)
    submit_button.click()

    time.sleep(1)

    # saving reports 

    results = driver.find_elements_by_xpath('//*[@id="content"]/table[5]/tbody/tr')

    violations = []

    for result in results[1:]: 

        data = {}
        if len(result.find_elements_by_class_name('drsviols')) != 0:
            print(result.find_elements_by_class_name('drsviols')[0].text)
            data['violator'] = result.find_elements_by_class_name('drsviols')[0].text
            #print(result.find_elements_by_class_name('drsviols')[1].text)
            data['contract_id'] = result.find_elements_by_class_name('drsviols')[1].text
            #print(result.find_elements_by_class_name('drsviols')[2].text)
            data['citation_no'] = result.find_elements_by_class_name('drsviols')[2].text
            #print(result.find_elements_by_class_name('drsviols')[3].text)
            data['case_no'] = result.find_elements_by_class_name('drsviols')[3].text
            #print(result.find_elements_by_class_name('drsviols')[4].text)
            data['date_issues'] = result.find_elements_by_class_name('drsviols')[4].text
            #print(result.find_elements_by_class_name('drsviols')[5].text)
            data['final_order_date'] = result.find_elements_by_class_name('drsviols')[5].text
            #print(result.find_elements_by_class_name('drsviols')[6].text)
            data['section_of_act'] = result.find_elements_by_class_name('drsviols')[6].text
            #print(result.find_elements_by_class_name('drsviols')[7].text)
            data['date_terminated'] = result.find_elements_by_class_name('drsviols')[7].text
            #print(result.find_elements_by_class_name('drsviols')[8].text)
            data['citation'] = result.find_elements_by_class_name('drsviols')[8].text
            #print(result.find_elements_by_class_name('drsviols')[9].text)
            data['s_and_s'] = result.find_elements_by_class_name('drsviols')[9].text
            #print(result.find_elements_by_class_name('drsviols')[10].text)
            
            try:
                #print(result.find_elements_by_class_name('drsviols')[10].text)
                data['standard'] = result.find_elements_by_class_name('drsviols')[10].text
                #print(result.find_elements_by_class_name('drsviols')[10].find_element_by_tag_name('a').get_attribute('href'))
                data['standard_url'] = result.find_elements_by_class_name('drsviols')[10].find_element_by_tag_name('a').get_attribute('href')
            except:
                print('url missing')
                data['standard'] = 'url missing'
                data['standard_url'] = 'url missing'

            try: 
                data['proposed_penalty'] = result.find_elements_by_class_name('drsviols')[11].text
                data['citation_status'] = result.find_elements_by_class_name('drsviols')[12].text
                data['current_penalty'] = result.find_elements_by_class_name('drsviols')[13].text
                data['amount_paid'] = result.find_elements_by_class_name('drsviols')[14].text
            except:
                data['proposed_penalty'] = 'error'
                data['citation_status'] = 'error'
                data['current_penalty'] = 'error'
                data['amount_paid'] = 'error'
                print('error')  
            
            violations.append(data)


    # saving the data

    df_final = pd.DataFrame(violations)
    df_final.to_csv(str(row['id'])+"-violations.csv", index=False)

In [84]:
df_mines.apply(get_mines_violations, axis=1)

4104757
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
Dirt Works
0801306
Holley Dirt Company, Inc
url missing
error
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
url missing
error
Holley Dirt Company, Inc
url missing
error
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley Dirt Company, Inc
Holley 

0    None
1    None
2    None
dtype: object

# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.

In [85]:
df_all_mines = pd.read_csv("mines.csv", dtype='str')
df_all_mines

,id
0,3503598
1,4801789
2,5001797
3,4608254
4,2103723
5,4104757
6,0801306
7,3901432
8,3609624
9,3609931


In [86]:
df_all_mines.apply(get_mines_violations, axis=1)

3503598
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
Newberg Rock & Dirt
4801789
AM D

url missing
error
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt Work LLC
R D Blankenship Dirt W

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object